In [42]:
import importlib
import moc_cart_threaded as MOC

# import moc_cart as MOC
import shapely 
from scipy.interpolate import griddata

import matplotlib.pyplot as plt

In [43]:
from plot_scalar_flux import *

In [44]:
materials = {
    "reflector": MOC.Material(2.0, 1.80, 0.0, "reflector", "blue"),
    "scatterer": MOC.Material(2.0, 1.99, 0.0, "scatterer", "red"),
    "absorber": MOC.Material(10.0, 2.0, 0.0, "absorber", "green"),
    "air": MOC.Material(0.1, 0.006, 0.0, "air", (0.9, 0.97, 1.0, 0.3)),
    "air2": MOC.Material(0.1, 0.09, 0.0, "air", (0.9, 0.97, 1.0, 0.3)),
    "isotropic": MOC.Material(0.1, 0, 1, "isotropic", "purple"),
    "detector": MOC.Material(1.0, 0, 0, "detector", "yellow"),
    "vacuum": MOC.Material(0.0, 0.0, 0.0, "vacuum", "black"),
}

In [45]:
total_dx = 5

def make_blocks(x0, y0, dx=0.1, dy=0.1, total_dx=total_dx, total_dy=total_dx, material=materials["reflector"]):
    """Create a grid of blocks in the x-y plane."""

    if type(material) is str:
        material = materials[material]

    number_blocks_x = int(total_dx / dx)
    assert number_blocks_x == total_dx / dx, "total_dx must be a multiple of dx"
    
    number_blocks_y = int(total_dy / dy)
    assert number_blocks_y == total_dy / dy, "total_dy must be a multiple of dy"

    cells = {}
    for i_block in range(number_blocks_x):
        for j_block in range(number_blocks_y):
            x = x0 + i_block * dx
            y = y0 + j_block * dy
            cell = shapely.geometry.box(x, y, x + dx, y + dy)
            cell_data = MOC.CellData(cell, material)
            cells[cell] = cell_data

    return cells


def plot_mesh(cells):
    # fig, ax = plt.subplots()
    # for cell, cell_data in cells.items():
    #     shapely.plotting.plot_polygon(
    #         cell, color=cell_data.material.color, alpha=0.5, ax=ax
    #     )

    fig, ax = plt.subplots()
    material_handles = {}
    for cell, cell_data in cells.items():
        coords = np.array(cell.exterior.coords)
        polygon = plt.Polygon(
            coords,
            closed=True,
            facecolor=cell_data.material.color,
            alpha=0.5,
            edgecolor="black",
            linewidth=0.03,
        )
        ax.add_patch(polygon)

        mat_name = cell_data.material.name
        if mat_name not in material_handles:
            material_handles[mat_name] = mpatches.Patch(
                facecolor=cell_data.material.color,
                edgecolor="black",
                alpha=0.5,
                label=mat_name,
            )

    ax.autoscale_view()
    ax.set_aspect("equal")

    ax.legend(
        handles=list(material_handles.values()),
        bbox_to_anchor=(1.05, 1),
        loc="upper left",
    )
    
    ax.set_xlabel("x (cm)")
    ax.set_ylabel("y (cm)")

    return fig, ax

In [46]:
dxs = [0.5, 0.2, 0.1]
total_dy = 15
dy = 1

# input_materials = ["air", "scatterer", "air", "isotropic", "air", "scatterer", "air"]
# lengths = [0.5, 0.5, 1, 0.5, 1, 0.5, 0.5]
input_materials = ["air"]
lengths = [5]
# starting_xs = np.cumsum(lengths) - lengths[0]
starting_xs = [0] + list(np.cumsum(lengths)[0:-1])


cells_per_dx = {}
for dx in dxs:
    cells = {}
    for input_material, length, starting_x in zip(input_materials, lengths, starting_xs):
        section_cells = make_blocks(
            starting_x, 0, dx=dx, dy=dy, total_dx=length, total_dy=total_dy, material=input_material
        )
        cells.update(section_cells)
    cells_per_dx[dx] = cells

# fig, ax = plot_mesh(cells)
# for ext in ["pdf", "svg"]:
#     fig.savefig(f"figures/03_mesh.{ext}", bbox_inches="tight")


In [47]:
importlib.reload(MOC)

quad = MOC.ProductQuadrature(8, 4)

low_res = MOC.CartesianMOC(cells_per_dx[0.5], quad, bc_west=1, ray_width=0.1)
med_res = MOC.CartesianMOC(cells_per_dx[0.2], quad, bc_west=1, ray_width=0.1)
high_res = MOC.CartesianMOC(cells_per_dx[0.1], quad, bc_west=1, ray_width=0.1)

low_res.solve()
med_res.solve()
high_res.solve()

Generating rays...
Rays generated in 2.95 seconds
Iter 0 error: 5.717363, time: 0.33 seconds
Iter 1 error: 0.147130, time: 0.05 seconds
Iter 2 error: 0.004287, time: 0.06 seconds
Iter 3 error: 0.000127, time: 0.05 seconds
Iter 4 error: 0.000004, time: 0.06 seconds
Iter 5 error: 0.000000, time: 0.06 seconds
Converged in 5 iterations with error 0.000000
Generating rays...
Rays generated in 1.02 seconds
Iter 0 error: 9.057903, time: 0.07 seconds
Iter 1 error: 0.232580, time: 0.07 seconds
Iter 2 error: 0.006779, time: 0.07 seconds
Iter 3 error: 0.000202, time: 0.07 seconds
Iter 4 error: 0.000006, time: 0.06 seconds
Iter 5 error: 0.000000, time: 0.07 seconds
Converged in 5 iterations with error 0.000000
Generating rays...
Rays generated in 1.47 seconds
Iter 0 error: 12.814520, time: 0.10 seconds
Iter 1 error: 0.328944, time: 0.09 seconds
Iter 2 error: 0.009589, time: 0.10 seconds
Iter 3 error: 0.000285, time: 0.08 seconds
Iter 4 error: 0.000009, time: 0.08 seconds
Iter 5 error: 0.000000, ti

In [48]:
def interpolate_to_ref_mesh(solution, reference_mesh):
    """Interpolate the scalar flux from the low resolution mesh to the high resolution mesh."""
    # Extract solution data from the solver
    cell_data = solution.cell_data_list
    centroids = [cell.cell.centroid for cell in cell_data]
    x = [c.x for c in centroids]
    y = [c.y for c in centroids]
    flux = [cell.prior_flux for cell in cell_data]

    ref_points = [
        (cell.cell.centroid.x, cell.cell.centroid.y)
        for cell in reference_mesh.cell_data_list
    ]
    # Linear interpolation
    interpolated_flux = griddata((x, y), flux, ref_points, method="linear")
    # Extrapolate using nearest for nan values
    nan_mask = np.isnan(interpolated_flux)
    if np.any(nan_mask):
        interpolated_flux[nan_mask] = griddata(
            (x, y), flux, np.array(ref_points)[nan_mask], method="nearest"
        )
    return np.array(interpolated_flux).flatten()

low_res_flux = interpolate_to_ref_mesh(low_res, high_res)
med_res_flux = interpolate_to_ref_mesh(med_res, high_res)

high_res_flux = [cell.prior_flux for cell in high_res.cell_data_list]

low_res_l2 = np.linalg.norm(low_res_flux - high_res_flux) / np.sqrt(
    len(high_res_flux)
)
print(f"L2 error low res: {low_res_l2:.4e}")

med_res_l2 = np.linalg.norm(med_res_flux - high_res_flux) / np.sqrt(
    len(high_res_flux)
)
print(f"L2 error med res: {med_res_l2:.4e}")

L2 error low res: 1.8247e-02
L2 error med res: 7.3216e-03
